In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-31.4"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 7)
val monthlyReportForecastUntil = YearMonthTO(2022, 1)

val yearlyReportFrom = 2019
val yearlyReportTo = 2020
val yearlyReportForecastUntil = 2022

val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=6b8cb67f-d872-471c-9ed1-59092cc7e2d5, username=Johann-31.4)

In [3]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE)))
reportView

ReportViewTO(id=6afd6166-9e56-4509-a98e-c95ea7193d0c, name=Expenses report, fundId=351feefa-6556-4e7f-bdd5-1da60a0a9911, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=false, filter=null), groupedNet=GenericReportConfigurationTO(enabled=false), grouped

In [4]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [5]:
val yearlyData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [ ]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(
    reportData: ReportDataTO,
    plotTitle: String,
    groupItemFilter: (ReportDataGroupedBudgetItemTO) -> Boolean,
): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }
                .reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values =
                    listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [ ]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

In [ ]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

In [ ]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

In [ ]:
plotGroup(monthlyReportData, "Home Expenses", "home")

In [ ]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

In [ ]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

In [ ]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

In [ ]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

In [ ]:
plotGroup(monthlyReportData, "Development Expenses", "development")

In [ ]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")